### Logistic回归分类器

#### 基于主要思想: 根据现有数据对分类边界线建立回归公式，以此进行分类
> 1. 优点：计算代价不高，以与理解和实现
> 2. 缺点：容易欠拟合，分类精度不高

#### 核心思想：
> 1. 首先确定分类器的基础函数，本例子为二类分类，所以采用在大尺度上可近似为Unit Step的Sigmoid函数   
Sigmoid: $f(z) = \frac{1}{1+e^{-z}}$
> 2. 回归: 对数据进行直线线性拟合的过程称为回归，即找到一条直线，使得数据点到直线的距离最小。在此例中，我们假设Sigmoid函数的输入为$z$，那么有:   
$z = \sum_{i=1}^{n}w_ix_i$，其中$w_i$为回归系数，$x_i$为输入数据，将$w_i$写作向量$\mathbf{w}$，则这个向量就是训练数据集的目标最佳回归系数向量。
> 3. 梯度上升法(最优化算法)   
假设函数$f(x,y)$，记其梯度为$\nabla f(x,y)$，该函数连续且处处可微，则其梯度为:   
$\nabla f(x,y) = \begin{pmatrix} \frac{\partial f}{\partial x} & \frac{\partial f}{\partial y} \end{pmatrix}^T$   
它表示要沿x轴方向移动$\frac{\partial f}{\partial x}$，沿y轴方向移动$\frac{\partial f}{\partial y}$
梯度上升算法到达每个点都会重新估计移动的方向，循环迭代直到满足停止条件，梯度算子总能保证选取的是最佳移动方向，而这个梯度算子在算法中只起到指引作用，还需要乘以步长值$\alpha$得到最终的移动量，那么梯度上升算法的迭代公式如下：   
$w:=w+\alpha\nabla_w f(w)$   
同理，对于梯度下降算法，其迭代公式为: $w:=w-\alpha\nabla_w f(w)$


In [ ]:
from numpy import *

def sigmoid(inX):
    """
    sigmoid函数
    """
    return 1.0 / (1 + exp(-inX))

def gradAscent(dataMatIn, classLabels):
    """
    梯度上升算法
    :param dataMatIn   数据集
    :param classLabels 数据集对应的类别
    """
    dataMatrix = mat(dataMatIn)
    labelMat = mat(classLabels).transpose()
    m, n = shape(dataMatrix)
    alpha = 0.001
    maxCycles = 500
    weights = ones((n, 1))
    for k in range(maxCycles):
        h = sigmoid(dataMatrix * weights)
        error = (labelMat - h)
        weights = weights + alpha * dataMatrix.transpose() * error
    return weights